In [62]:
'Shanzay Asad - 19I1998'
'AI Assignment 2'

'AI Assignment 2'

In [63]:
#Importing all valid libraries to be used for this assignment
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [64]:
data = pd.read_fwf("adult.data") 
test = pd.read_fwf("adult.test")
test

,|1x3 Cross validator
0,"25, Private, 226802, 11th, 7, Never-married, M..."
1,"38, Private, 89814, HS-grad, 9, Married-civ-sp..."
2,"28, Local-gov, 336951, Assoc-acdm, 12, Married..."
3,"44, Private, 160323, Some-college, 10, Married..."
4,"18, ?, 103497, Some-college, 10, Never-married..."
...,...
16276,"39, Private, 215419, Bachelors, 13, Divorced, ..."
16277,"64, ?, 321403, HS-grad, 9, Widowed, ?, Other-r..."
16278,"38, Private, 374983, Bachelors, 13, Married-ci..."
16279,"44, Private, 83891, Bachelors, 13, Divorced, A..."


In [65]:
#Preprocessing starts here
data.rename(columns = {'State-gov, 77516, Bachelors, 13, Never-married, Adm-clerical, Not-in-family, White, Male, 2174, 0, 40, United-States, <=50K':
'Characteristics'}, inplace = True)

test.rename(columns = {'|1x3 Cross validator':'Characteristics'}, inplace = True)

test.head()

,Characteristics
0,"25, Private, 226802, 11th, 7, Never-married, M..."
1,"38, Private, 89814, HS-grad, 9, Married-civ-sp..."
2,"28, Local-gov, 336951, Assoc-acdm, 12, Married..."
3,"44, Private, 160323, Some-college, 10, Married..."
4,"18, ?, 103497, Some-college, 10, Never-married..."


In [66]:
char = data['39,']+ ' ' + data['Characteristics']
char = char.str.split(",")

char_t = test['Characteristics'].str.split(",")
#print(char_t)
lis = []
lis_t = []
for i in range(0, len(char)):
    lis.append(str(char[i][-1:])[2:-2].strip())

for i in range(0, len(char_t)):
    lis_t.append(str(char_t[i][-1:])[2:-3].strip()) 


In [67]:
data['Characteristics'] = char
data['Characteristics'] = data['Characteristics'].apply(lambda x: ' '.join(map(str,x)))
data['Salary'] = lis

test['Characteristics'] = char_t.apply(lambda x: ' '.join(map(str,x)))
test['Salary'] = lis_t

print(test)
#print(data)

                                         Characteristics Salary
0      25  Private  226802  11th  7  Never-married  M...  <=50K
1      38  Private  89814  HS-grad  9  Married-civ-sp...  <=50K
2      28  Local-gov  336951  Assoc-acdm  12  Married...   >50K
3      44  Private  160323  Some-college  10  Married...   >50K
4      18  ?  103497  Some-college  10  Never-married...  <=50K
...                                                  ...    ...
16276  39  Private  215419  Bachelors  13  Divorced  ...  <=50K
16277  64  ?  321403  HS-grad  9  Widowed  ?  Other-r...  <=50K
16278  38  Private  374983  Bachelors  13  Married-ci...  <=50K
16279  44  Private  83891  Bachelors  13  Divorced  A...  <=50K
16280  35  Self-emp-inc  182148  Bachelors  13  Marri...   >50K

[16281 rows x 2 columns]


In [68]:

data['Characteristics'] = data['Characteristics'].str.lower()
test['Characteristics'] = test['Characteristics'].str.lower()
from string import digits
data['Characteristics'] = data['Characteristics'].str.translate({ord(k): None for k in digits}) #Removing alphanumeric from data with lambda function
test['Characteristics'] = test['Characteristics'].str.translate({ord(k): None for k in digits}) #Removing alphanumeric from data with lambda function

#Finally dropping the coloumn we don't require
data = data.drop(columns=['39,'])
data = data.dropna() #We do this to remove any nulls or undefined data points in our data
test = test.dropna()

In [69]:
test.head()

,Characteristics,Salary
0,private th never-married machine-op-i...,<=50K
1,private hs-grad married-civ-spouse fa...,<=50K
2,local-gov assoc-acdm married-civ-spous...,>50K
3,private some-college married-civ-spous...,>50K
4,? some-college never-married ? own-c...,<=50K


In [70]:
lis = ['?']
repeated = ['>50K', '<=50K'] # means the same thing therefore it can be removed
test['Characteristics'] = test['Characteristics'].str.translate({ord(k): None for k in lis})  #This is removing the ? we see
data['Characteristics'] = data['Characteristics'].str.translate({ord(k): None for k in lis})  #This is removing the ? we see

data = data[data["Salary"].str.contains("<=50K")==False]
data = data[data["Salary"].str.contains('>50K')==False]


test = test[test["Salary"].str.contains("<=50K")==False]
test = test[test["Salary"].str.contains('>50K')==False]

test.head()

,Characteristics,Salary
243,self-emp-not-inc some-college married-...,<=50
259,self-emp-not-inc prof-school never-mar...,<=5
298,self-emp-not-inc assoc-voc never-marri...,>50
357,self-emp-not-inc prof-school married-c...,>50
376,private bachelors married-civ-spouse ...,<=5


In [71]:
label = LabelEncoder()
y_train = label.fit_transform(data['Salary'])
y_test = label.fit_transform(test['Salary'])

#Giving label to Salary makes it easier to predict it as a feature (as y)

In [72]:
vec = CountVectorizer(stop_words='english').fit(data['Characteristics'])
x_train = vec.transform(data['Characteristics'])

x_test = vec.transform(test['Characteristics'])


In [73]:
x_train

<766x115 sparse matrix of type '<class 'numpy.int64'>'
	with 10001 stored elements in Compressed Sparse Row format>

In [74]:
y_train.shape

(766,)

In [75]:
x_test

<191x115 sparse matrix of type '<class 'numpy.int64'>'
	with 2561 stored elements in Compressed Sparse Row format>

Logistic Regression

In [76]:
y_test.shape

(191,)

In [1]:
#x_train = x_train.reshape(32560,125)
LR = LogisticRegression().fit(x_train, y_train) #Loading LR into a valid object for further use and abbreviation
LR.predict(x_test)

print(str(LR.score(x_test, y_test)*100) + "% accuracy")


NameError: name 'LogisticRegression' is not defined

Naive Bayes

In [78]:
mnb = MultinomialNB()
mnb.fit(x_train, y_train)

mnb.predict(x_test)
print(str(mnb.score(x_test, y_test)*100) + "% accuracy")

12.041884816753926% accuracy


For some reason, the accuracy being reached is drastically low which can be chocked up to a issue with preprocessing. One possible issue identified is not removing repeated values
However, by general concensus and by seeing the printing accuracy we can determine that the Logistic Regression model has given us a better accuracy